### Postprocessing Results

In [26]:
import glob as gb
import ntpath # Equivalent of R's base (remove path)
import pandas as pd

In [31]:
# Specify the folder where the results are located
BUCKET = 'bucket1'
local_dir = f'/Users/jasonpayne/Documents/slu_res/{BUCKET}/'
plot_files = gb.glob(f'{local_dir}*.mplt')

In [32]:
# Create a list of load cases that are covered by the timetrace files
lc_files = []
for file_p in plot_files:
    lc_files.append(ntpath.basename(file_p[:len(file_p) - 8]))
    
lc_files = list(dict.fromkeys(lc_files))
lc_files.sort()

In [33]:
# Create empty lists of take processed results
load_cases = []
out_verDisp_mm_n5 = []
out_oopDisp_mm_n5 = []
out_oopDisp_mm_n2 = []
out_endBend_kNm_n6 = []
out_supReac_kN_n3 = []
out_supReac_kN_n4 = []

# Loop through and process each load case and each plot file in each load case
for lc in lc_files:
    load_cases.append(lc)
    for tt in ['T1', 'T3', 'T4', 'T5', 'T6', 'T7']:
        # Form the filename from the load case reference
        file_p = f'{lc}.{tt}.mplt'
        
        # Download blob as StorageStreamDownloader object (stored in memory)
        plot_raw = pd.read_csv(f'{local_dir}{file_p}', skiprows=12, delimiter='"')
        dat_dt = {'time':plot_raw.iloc[:,3].dropna(), 'value':plot_raw.iloc[:,5].dropna()}
        dat_df = pd.DataFrame(dat_dt)

        # Breakdown of types of timetrace
        motions = ['T1', 'T2', 'T3', 'T4']
        moments = ['T5', 'T6']
        reactin = ['T7']

        # Some unit conversions depending on type
        # Use mm for displacement ranges
        if (any(x in file_p for x in motions)):
            dat_df.value = dat_df.value*1000

        elif (any(x in file_p for x in moments)):
            dat_df.value = dat_df.value/1000

        elif (any(x in file_p for x in reactin)):
            dat_df.value = dat_df.value/1000

        # Calculate the range
        dat_range = dat_df.value.max() - dat_df.value.min()
        print(file_p[:len(file_p) - 8])
        print(f'Max: {round(dat_df.value.max(), 1)}, Min: {round(dat_df.value.min(), 1)}') 
        print('Range: ', round(dat_range, 1), '\n')

        # Collect the results
        if 'T1' in file_p:
            out_verDisp_mm_n5.append(round(dat_range, 2))
        elif 'T3' in file_p:
            out_oopDisp_mm_n5.append(round(dat_range, 2))
        elif 'T4' in file_p:
            out_oopDisp_mm_n2.append(round(dat_range, 2))
        elif 'T5' in file_p:
            out_endBend_kNm_n6.append(round(dat_range, 2))
        elif 'T6' in file_p:
            out_supReac_kN_n3.append(round(dat_range, 2)) 
        elif 'T7' in file_p:
            out_supReac_kN_n4.append(round(dat_range, 2))     
    

In [30]:
# Now compile the results in a single dataframe
res_df = pd.DataFrame(load_cases, columns = ['load_case'])
res_df['out_oopDisp_mm_n2'] = out_oopDisp_mm_n2
res_df['out_supReac_kN_n3'] = out_supReac_kN_n3
res_df['out_supReac_kN_n4'] = out_supReac_kN_n4
res_df['out_verDisp_mm_n5'] = out_verDisp_mm_n5
res_df['out_oopDisp_mm_n5'] = out_oopDisp_mm_n5
res_df['out_endBend_kNm_n6'] = out_endBend_kNm_n6
res_df

ValueError: Length of values (142) does not match length of index (143)

In [ ]:
# Write the results as a csv
res_df.to_csv(f'{BUCKET}_results.csv', index=False)